In [116]:
import numpy as np
import pandas as pd

# Define the range of wind speeds and the number of states
wind_speed_range = (0, 50)
wind_speed_states = 50
wind_speed_interval = (wind_speed_range[1] - wind_speed_range[0]) / wind_speed_states
wind_speed_states_list = [wind_speed_range[0] + i * wind_speed_interval for i in range(wind_speed_states)]

# Define the range of angles of attack and the number of states
angle_range = (-15, 15)
angle_states = 30
angle_interval = (angle_range[1] - angle_range[0]) / angle_states
angle_states_list = [angle_range[0] + i * angle_interval for i in range(angle_states)]

# Define the number of actions
actions = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5] # angle change

#define wind speed change
wind_speed_change = 1

counter = 0 #counts the number of times it takes for algorithm to keep the lift constant within loop


In [117]:
# Define the function to calculate the lift of the airfoil
def lift(angle_of_attack, wind_speed):
    # Calculate the Reynolds number
    air_density = 1.225 # kg/m^3 (sea level)
    air_viscosity = 1.78e-5 # Ns/m^2 (sea level)
    chord_length = 0.15 # m (geometry of the NACA 0015 airfoil)
    reynolds_number = wind_speed * chord_length / air_viscosity
    
    # Calculate the lift coefficient
    C_l_alpha = 2 * np.pi / 180
    C_l_0 = 0.0060
    lift_coefficient = C_l_alpha * angle_of_attack + C_l_0
    
    # Calculate the lift force
    lift_force = 0.5 * lift_coefficient * air_density * wind_speed ** 2 * chord_length
    
    return lift_force

In [118]:
# Miscellaneous functions to aid the functioning of the code
def find_closest_index(value, list_values):
    closest_index = 0
    closest_value = float("inf")
    for i, list_value in enumerate(list_values):
        if abs(value - list_value) < closest_value:
            closest_value = abs(value - list_value)
            closest_index = i
    return closest_index

In [119]:
# Function to convert a continuous angle of attack to its corresponding discrete value
def get_discrete_value(value, values):
    return np.argmin(np.abs(np.array(values) - value))

In [131]:
# Initialize the table
table = []
headers = ['Wind Speed', 'Angle', 'Lift', 'Reward', 'counter']
table.append(headers)

# Define the Q-table and some initial parameters
q_table = np.zeros((wind_speed_states, angle_states, len(actions)))
alpha = 0.1
gamma = 0.9
epsilon = 0.1
# maximum number of episodes
max_episodes = 1000
max_steps = 100

In [132]:
def rewards(state, action, next_state, goal_state):
    lift_threshold = 0.1  # within 10% of desired value
    better_threshold = 0.01 # within 1% of desired value
    
    if next_state == goal_state:
        return 10 # high reward for reaching the goal
    else:
        diff_lift = abs(next_state - goal_state)
        if diff_lift > lift_threshold:
            return -1 # penalty for not reaching the goal
        elif better_threshold < diff_lift < lift_threshold:
            return 1 # neutral reward for being close to the goal
        elif diff_lift < better_threshold:
            return 3

In [133]:
# Define the Q-learning algorithm
for episode in range(max_episodes):
    # Initialize the state and the lift
    current_wind_speed = 1 # m/s
    current_angle = 10 # degrees
    current_lift = lift(current_angle, current_wind_speed)
    goal_lift = lift(10,1)
    counter = 0
        
    # Run the Q-learning algorithm
    for _ in range(max_steps):
        counter += 1
        
        new_wind_speed = current_wind_speed + wind_speed_change
        # Check if the new wind speed is within the range of possible wind speeds
        if new_wind_speed < wind_speed_range[0]:
            new_wind_speed = wind_speed_range[0]
        elif new_wind_speed >= wind_speed_range[1]:
            new_wind_speed = wind_speed_range[1] - 1
        
        # Choose an action based on the Q-table and epsilon-greedy strategy
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(0, len(actions))
        else:
            action = np.argmax(q_table[current_wind_speed, current_angle, :])
        
        # Calculate the new lift and the new state
        new_angle = current_angle + actions[action]
        
        if new_angle < angle_range[0]:
            new_angle = angle_range[0]
        elif new_angle >= angle_range[1]:
            new_angle = angle_range[1] - 1
        
        new_lift = lift(new_angle, new_wind_speed)
        current_lift = lift(current_angle, current_wind_speed)
        
        # Reward Calculation
        reward = rewards(current_lift, action, new_lift, goal_lift)
        
        # Last check for wind speed and angle
        if new_wind_speed >= wind_speed_states:
            new_wind_speed = wind_speed_states - 1
        elif new_wind_speed < 0:
            new_wind_speed = 0
        
        # Update Q-table
        q_table[current_wind_speed, current_angle, action] = \
        (1 - alpha) * q_table[current_wind_speed, current_angle, action] + \
        alpha * (reward + gamma * np.max(q_table[new_wind_speed, new_angle, :]))
        
        # Update the current state to the new state
        current_wind_speed = new_wind_speed
        current_angle = new_angle
        epsilon = epsilon - 0.001
    
        # Print some output to track progress
        table.append([current_wind_speed, current_angle, current_lift, reward, counter])   
    if episode % 100 == 0:
        print(f"Episode {episode} completed.")


Episode 0 completed.
Episode 100 completed.
Episode 200 completed.
Episode 300 completed.
Episode 400 completed.
Episode 500 completed.
Episode 600 completed.
Episode 700 completed.
Episode 800 completed.
Episode 900 completed.


In [134]:
q_table

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [135]:
table

[['Wind Speed', 'Angle', 'Lift', 'Reward', 'counter'],
 [2, 5, 0.032621675005395805, 1, 1],
 [3, 0, 0.06634585001079162, 1, 2],
 [4, -5, 0.0049612499999999995, -1, 3],
 [5, -10, -0.24774340004316642, -1, 4],
 [6, -15, -0.7879793751348952, -1, 5],
 [7, -15, -1.7119579502913735, -1, 6],
 [8, -15, -2.3301649878965915, -1, 7],
 [9, -15, -3.043480800517997, -1, 8],
 [10, -15, -3.8519053881555902, -1, 9],
 [11, -15, -4.755438750809371, -1, 10],
 [12, -15, -5.754080888479338, -1, 11],
 [13, -15, -6.847831801165494, -1, 12],
 [14, -15, -8.036691488867836, -1, 13],
 [15, -15, -9.320659951586366, -1, 14],
 [16, -15, -10.699737189321084, -1, 15],
 [17, -15, -12.173923202071988, -1, 16],
 [18, -15, -13.74321798983908, -1, 17],
 [19, -15, -15.407621552622361, -1, 18],
 [20, -15, -17.167133890421827, -1, 19],
 [21, -15, -19.021755003237484, -1, 20],
 [22, -15, -20.971484891069327, -1, 21],
 [23, -15, -23.016323553917353, -1, 22],
 [24, -15, -25.15627099178157, -1, 23],
 [25, -15, -27.391327204661977

In [70]:
goal_lift = lift(15,20)
print(goal_lift)

19.462755003237483


In [86]:
len(q_table[0][0])

11